# 1. Import Dependencies

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

## How to get new data
Remember to do a backup before to proceed, look at backup\appunti.txt.

To generate new data pen scripts\getDataHandGesture.py and edit labels adding new gestures. 

Execute it to export data in src\dataHandGesture

## How to export and use the generated model
At the end of the execution the generated model will be here: Tensorflow\workspace\models

Then, copy the name of the folder inside Tensorflow\workspace\models\my_hand_gesture_model\ (not eval, should be something like 1642800231) and paste in
scripts\handGestureModule.py in the first variable named "lastModel"

## Dataset
This specific dataset seperates hand gestures into 10 different classes.

<img style="float: center;" src="document\images\gestures.png">

- backward:
    > this command allows the drone to go backwards;
- detect:
    > this command allows you to detect a 3d trajectory;
- down: 
    > this command allows the drone to go down;
- forward: 
    > this command allows the drone to go forward;
- land:
    > this command allows the drone to land;
- left:
    > this command allows the drone to go left;
- ok:
    > this command allows the drone to go close and execute a 3d trajectory;
- right:
    > this command allows the drone to go right;
- stop:
    > this command allows the drone to stop his movements;
- up:
    > this command allows the drone to go up.

The information about each hand gesture is contained into 42 features.

In [ ]:
# CSV_COLUMN_NAMES = np.arange(43)
# CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]

CSV_COLUMN_NAMES = ["WRIST_X", "WRIST_Y",
                    "THUMB_CMC_X", "THUMB_CMC_Y",
                    "THUMB_MCP_X", "THUMB_MCP_Y",
                    "THUMB_IP_X", "THUMB_IP_Y",
                    "THUMB_TIP_X", "THUMB_TIP_Y",
                    "INDEX_FINGER_MCP_X", "INDEX_FINGER_MCP_Y",
                    "INDEX_FINGER_PIP_X", "INDEX_FINGER_PIP_Y",
                    "INDEX_FINGER_DIP_X", "INDEX_FINGER_DIP_Y",
                    "INDEX_FINGER_TIP_X", "INDEX_FINGER_TIP_Y",
                    "MIDDLE_FINGER_MCP_X", "MIDDLE_FINGER_MCP_Y",
                    "MIDDLE_FINGER_PIP_X", "MIDDLE_FINGER_PIP_Y",
                    "MIDDLE_FINGER_DIP_X", "MIDDLE_FINGER_DIP_Y",
                    "MIDDLE_FINGER_TIP_X", "MIDDLE_FINGER_TIP_Y",
                    "RING_FINGER_MCP_X", "RING_FINGER_MCP_Y",
                    "RING_FINGER_PIP_X", "RING_FINGER_PIP_Y",
                    "RING_FINGER_DIP_X", "RING_FINGER_DIP_Y",
                    "RING_FINGER_TIP_X", "RING_FINGER_TIP_Y",
                    "PINKY_MCP_X", "PINKY_MCP_Y",
                    "PINKY_PIP_X", "PINKY_PIP_Y",
                    "PINKY_DIP_X", "PINKY_DIP_Y",
                    "PINKY_TIP_X", "PINKY_TIP_Y",
                    "TARGET"]

TARGET = ['backward', 'detect', 'down', 'forward', 'land', 'left', 'ok', 'right', 'stop', 'up']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [ ]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [ ]:
train.head()

In [ ]:
train_y = train.pop('TARGET')
test_y = test.pop('TARGET')
train_y = train_y.apply(np.int32)
test_y = test_y.apply(np.int32)
train.head() # the TARGET column is now gone

In [ ]:
train.shape  # we have 800 entires with 42 features

EDA
We'll just do a few quick plots of the data.

In [ ]:
import seaborn as sns # data visualization library  

In [ ]:
sns.set(style="white")
x = df.loc[:,['WRIST_X','1','2']]
g = sns.PairGrid(x, diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot, lw=3)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [ ]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model_test")

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between len(TARGET) classes.
    n_classes=len(TARGET),
    activation_fn=lambda x: tf.nn.leaky_relu(x, alpha=0.01),
    optimizer=lambda: tf.keras.optimizers.Adam(
           learning_rate=tf.compat.v1.train.exponential_decay(
               learning_rate=0.1,
               global_step=tf.compat.v1.train.get_global_step(),
               decay_steps=10000,
               decay_rate=0.96)
       ),
    model_dir= MODEL_PATH)

## Training

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=3000)

## Export Model

In [ ]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns)
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True)

## Load Model

In [ ]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

In [ ]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

In [ ]:
# https://stackoverflow.com/questions/46766606/how-i-get-accurcy-graphnot-dot-like-loss-graph-in-tensorboard

# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

In [ ]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {TARGET[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {TARGET[test_y.iloc[idx]]}")

## Evaluation

In [ ]:
# tensorboard --logdir .
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False), steps=1)

In [ ]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
eval_result

## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = TARGET[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            TARGET[class_id], 100 * probability))
        print(f"Expected is {expected}")

# (NEW ADDED 26/01/2022) Evaluating model performance

https://www.kaggle.com/riteshsinha/tensorflow-2-classifying-tumors-deep-learning

https://www.kaggle.com/karthickaravindan/dnnclassifier

In [ ]:
# Prediction is done here now.
predictions = list(predict_fn(examples=examples)["class_ids"].numpy().T[0])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(test_y, predictions)
print(cm)

In [ ]:
# https://stackoverflow.com/questions/67636940/confusionmatrixdisplay-scikit-learn-plot-labels-out-of-range
cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=TARGET)
fig, ax = plt.subplots(figsize=(15,15)) # set size
cmp.plot(ax=ax, xticks_rotation=45)

SMALL_SIZE = 30
MEDIUM_SIZE = 40
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.show()

In [ ]:
print(classification_report(test_y, predictions))

In [ ]:
accuracy_score(test_y, predictions, normalize=True, sample_weight=None)